In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import lxml
import bs4
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set up Selenium WebDriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (no GUI)
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [4]:
url = "https://fbref.com/en/comps/9/gca/Premier-League-Stats"

In [5]:
driver.get(url)

In [6]:
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "stats_gca"))
    )
except:
    print("❌ The table did not load.")
    driver.quit()
    exit()


In [7]:
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()  # Close the browser after getting the HTML

In [8]:
table = soup.find("table", {"id": "stats_gca"})
#table = tables[1]
if not table:
    print("❌ The table with ID 'stats_gca' was not found.")
    exit()

In [9]:
players_data = []
for row in table.find("tbody").find_all("tr"):  # Get rows inside tbody
    player_cell = row.find("td", {"data-stat": "player"})  # Find player cell
    if player_cell:
        player_name = player_cell.text.strip()
        player_link = player_cell.find("a")  # Find hyperlink inside <td>
        if player_link:
            player_url = "https://fbref.com" + player_link["href"]
            players_data.append([player_name, player_url])

In [10]:
df = pd.DataFrame(players_data, columns=["Player", "URL"])

In [11]:
df.to_csv("players_list_gca.csv", index=False)
print("✅ Player list saved to players_list_gca.csv")

✅ Player list saved to players_list_gca.csv


In [13]:
df

,Player,URL
0,Max Aarons,https://fbref.com/en/players/774cf58b/Max-Aarons
1,Joshua Acheampong,https://fbref.com/en/players/b7e62e1d/Joshua-A...
2,Tyler Adams,https://fbref.com/en/players/2b09d998/Tyler-Adams
3,Tosin Adarabioyo,https://fbref.com/en/players/c81d773d/Tosin-Ad...
4,Simon Adingra,https://fbref.com/en/players/4dcec659/Simon-Ad...
...,...,...
544,Ashley Young,https://fbref.com/en/players/be927d03/Ashley-Y...
545,Illia Zabarnyi,https://fbref.com/en/players/88968486/Illia-Za...
546,Oleksandr Zinchenko,https://fbref.com/en/players/51cf8561/Oleksand...
547,Joshua Zirkzee,https://fbref.com/en/players/028e70b9/Joshua-Z...
